# Simulation on flipping noise (Figure 5)

This notebook contains the code for the simulation with flipping noise on COMPAS 

In [ ]:
%load_ext autoreload
%autoreload 2

# general imports
import numpy as np
import sys, random
import os, warnings
import itertools as iter
from copy import deepcopy
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# initialization
rng = np.random.default_rng(1234)
random.seed()

# add to path
print(os.getcwd())
sys.path.append(os.getcwd())

# import ai360
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, CompasDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas

# import main code
from utils import *
import algorithms as denoisedfair
from lamy_noise_fairlearn.util import *
from awasthi_equalized_odds_under_perturbation.equalized_odds import *

from scipy.optimize import minimize

In [ ]:
exec(open('baseSimulation.py').read())

In [ ]:
result_folder = 'results'
os.system(f'mkdir {result_folder}')

## Simulation code

In [ ]:
# which algorithms to skip?
skipUnconstrained = False

skipErrTolerant = False
skipLamy =  False
skipAwasthi =  False
skipKL21 =  False
skipDenoised = False

assert(skipAwasthi or not skipUnconstrained)

runParallel = True

## Running the simulation

In [ ]:
reps = 100
CORES = 10
flip_func = flipping

In [ ]:
all_results = {}
for e1 in tqdm([0.0, 0.035]):
    for e2 in tqdm([0.035, 0.0]):
        st = time.time()
        
        eta = [e1, e2]
        all_results[(e1,e2)] = test_predict_product(eta, flip_func, reps=reps, CORES=CORES, metric='sr', get_eta_avg=True)
        
        print(f'Time per loop: {time.time() - st} seconds')
        
        index = {0.035: 1, 0: 0}
        f = open(f'{result_folder}/StocNoise-{index[e1]}{index[e2]}-{reps}iter', 'wb')
        pickle.dump(all_results, f)